In [62]:
import pandas as pd
import numpy as np
import json
import requests

In [63]:
print('\n\n\n=======================\nALGORITHMIC TRADING BOT\
    \nTEST ENVIRONMENT\n=======================\n\n\n')

balance = 10000
print(f'Starting Balance: ${balance}')

print('\nAlgorithm Rules:\nTrade Size = 5% of balance.\nTP/SL = 1 ATR\nDaily Profit Target = 2% Profit')




ALGORITHMIC TRADING BOT    
TEST ENVIRONMENT



Starting Balance: $10000

Algorithm Rules:
Trade Size = 5% of balance.
TP/SL = 1 ATR
Daily Profit Target = 2% Profit


In [64]:
# import datetime
starting_time = 1669852800000
limit = 1000
timeframe = 60 #in minutes

def convert_minutes_to_milliseconds(minutes):
    milliseconds = 60000 * minutes
    return milliseconds


url1 = f'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit={limit}&endTime={starting_time}'
url2 = f'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit={limit}&endTime={starting_time - limit*(convert_minutes_to_milliseconds(timeframe))}'

urls = [url1, url2]

In [65]:
def get_bars(symbol):
    df = pd.DataFrame()
    for i in urls:
        data = json.loads(requests.get(i).text)
        print(i)
        for row in data:
            del row[5:]
        df = df.append(data)
    df.columns = ['date',
                'open', 'high', 'low', 'close']
    return df

eth_df_raw = get_bars('ETHUSDT')
eth_df_raw = eth_df_raw.sort_values(by='date')

https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1669852800000


/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_88430/3063555294.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1666252800000


In [66]:
%%script echo skipping

eth_df_raw['open'] = pd.to_numeric(eth_df_raw['open'])
eth_df_raw['high'] = pd.to_numeric(eth_df_raw['high'])
eth_df_raw['low'] = pd.to_numeric(eth_df_raw['low'])
eth_df_raw['close'] = pd.to_numeric(eth_df_raw['close'])
eth_df_raw['date'] = pd.to_datetime(eth_df_raw['date'], unit = 'ms')

skipping


In [67]:
%%script echo skipping
eth_df_raw

skipping


In [68]:
def test_logic(df):
    df['open'] = pd.to_numeric(df['open'])
    df['high'] = pd.to_numeric(df['high'])
    df['low'] = pd.to_numeric(df['low'])
    df['close'] = pd.to_numeric(df['close'])
    df['date'] = pd.to_datetime(df['date'], unit = 'ms')


    short_ema = df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    df['macd'] = macd
    df['signal'] = signal
    df['200_ema'] = ema_200

    df['TR1'] = abs(df['high'] - df['low'])
    df['TR2'] = abs(df['low'] - df['close'].shift())
    df['TR3'] = abs(df['high'] - df['close'].shift())

    df['ATR'] = df.apply(lambda x: row_check(x.TR1, x.TR2, x.TR3), axis=1)

    df = df.iloc[200:]

    df = df.drop(['TR1', 'TR2', 'TR3'], axis=1)

    df['trigger_long'] = np.where(((df['macd'] > df['signal']) & (df['close'] > df['200_ema']) & (df['macd'] < 0) & (df['signal'] > df['macd'].shift())), 1, 0)

    df['trigger_short'] = np.where(((df['signal'] > df['macd']) & (df['close'] < df['200_ema']) & (df['macd'] > 0) & (df['signal'] < df['macd'].shift())), 1, 0)


    return df

def row_check(TR1, TR2, TR3):
    if TR1 >= TR2 and TR1 >= TR3:
        ATR = TR1
    elif TR2 >= TR1 and TR2 >= TR3:
        ATR = TR2
    else:
        ATR = TR3
    
    return ATR

In [69]:
eth_cleaned = test_logic(eth_df_raw)

In [70]:
eth_cleaned

,date,open,high,low,close,macd,signal,200_ema,ATR,trigger_long,trigger_short
200,2022-09-17 01:00:00,1443.05,1448.41,1438.38,1445.64,-17.207603,-21.662962,1600.555756,10.03,0,0
201,2022-09-17 02:00:00,1445.63,1446.23,1436.77,1439.71,-16.057287,-20.541827,1598.955301,9.46,0,0
202,2022-09-17 03:00:00,1439.72,1442.38,1436.72,1440.57,-14.904450,-19.414352,1597.379328,5.66,0,0
203,2022-09-17 04:00:00,1440.57,1447.92,1439.23,1440.87,-13.807447,-18.292971,1595.822021,8.69,0,0
204,2022-09-17 05:00:00,1440.87,1440.88,1425.09,1432.06,-13.493413,-17.333059,1594.192548,15.79,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,2022-11-30 20:00:00,1287.20,1295.76,1285.00,1295.29,14.258507,13.990074,1209.811179,10.76,0,0
996,2022-11-30 21:00:00,1295.29,1299.00,1287.73,1297.07,15.059045,14.203868,1210.679426,11.27,0,0
997,2022-11-30 22:00:00,1297.07,1309.77,1292.24,1298.91,15.661413,14.495377,1211.557342,17.53,0,0
998,2022-11-30 23:00:00,1298.90,1306.51,1290.84,1294.46,15.599892,14.716280,1212.382244,15.67,0,0


In [71]:
eth_cleaned.loc[eth_cleaned['trigger_short'] == 1]

,date,open,high,low,close,macd,signal,200_ema,ATR,trigger_long,trigger_short
230,2022-09-18 07:00:00,1454.37,1457.49,1452.56,1453.45,1.687028,1.807193,1561.014153,4.93,0,1
280,2022-09-20 09:00:00,1357.50,1359.61,1345.14,1352.57,1.651947,1.718058,1480.308063,14.47,0,1
282,2022-09-20 11:00:00,1360.27,1365.27,1353.35,1356.26,1.457044,1.668190,1477.891131,11.92,0,1
288,2022-09-20 17:00:00,1367.31,1368.99,1332.29,1335.43,0.211579,1.110978,1470.766888,36.70,0,1
352,2022-09-23 09:00:00,1317.51,1322.97,1303.80,1312.98,7.953044,8.617565,1396.281396,19.17,0,1
353,2022-09-23 10:00:00,1312.97,1316.07,1290.00,1294.80,4.951073,7.884266,1395.271630,26.07,0,1
388,2022-09-24 21:00:00,1341.99,1342.39,1319.42,1322.96,5.875134,6.800464,1373.258424,22.97,0,1
494,2022-09-29 07:00:00,1328.58,1330.53,1313.55,1322.00,3.277441,4.256251,1342.753992,16.98,0,1
514,2022-09-30 03:00:00,1326.38,1329.61,1320.12,1327.49,0.940613,1.206934,1340.366832,9.49,0,1
523,2022-09-30 12:00:00,1330.79,1343.50,1313.35,1327.48,1.451912,1.875953,1339.931016,30.15,0,1


In [72]:
eth_cleaned.loc[eth_cleaned['trigger_long'] == 1]

,date,open,high,low,close,macd,signal,200_ema,ATR,trigger_long,trigger_short
649,2022-10-05 18:00:00,1348.51,1357.39,1342.96,1357.39,-0.166418,-0.478901,1331.309333,14.43,1,0
689,2022-10-07 10:00:00,1357.02,1357.30,1353.09,1356.59,-0.933010,-0.954220,1340.643121,4.21,1,0
196,2022-10-28 13:00:00,1511.46,1549.94,1509.32,1540.18,-6.358376,-8.290087,1424.038777,40.62,1,0
260,2022-10-31 05:00:00,1589.44,1596.21,1587.50,1589.16,-5.465328,-5.504537,1505.573210,8.71,1,0
261,2022-10-31 06:00:00,1589.17,1592.16,1578.41,1586.85,-5.073418,-5.418313,1506.381934,13.75,1,0
264,2022-10-31 09:00:00,1582.62,1623.00,1579.45,1620.76,-1.407182,-4.324659,1509.098021,43.55,1,0
280,2022-11-01 01:00:00,1570.11,1584.37,1569.68,1581.18,-6.254078,-6.990389,1518.964857,14.69,1,0
303,2022-11-02 00:00:00,1578.47,1592.44,1572.09,1587.54,-1.390156,-1.513235,1532.473857,20.35,1,0
304,2022-11-02 01:00:00,1587.55,1591.06,1585.33,1586.01,-0.839046,-1.378397,1533.006555,5.73,1,0
329,2022-11-03 02:00:00,1543.19,1549.99,1542.08,1543.59,-11.855933,-11.923407,1537.567148,7.91,1,0


In [73]:
in_long = False
in_short = False

tp1 = 0
tp2 = 0

tp1_hit = False

trades = 0

for index,row in eth_cleaned.iterrows():

    if in_long == False and in_short == False:

        if row['trigger_long'] == 1:

            print('LONG')

            atr = round(row['ATR'],2)
            entry = round(row['close'],2)
            stop_loss = round(entry - (1*atr),2)
            tp1 = round(entry + atr,2)
            tp2 = round(entry + 2*atr,2)

            balance_split = balance*0.05
            amount_bought = (balance_split)/entry
            balance -= balance_split

            in_long = True

            print(f'ATR: ${atr}\nEntry: ${entry}\nStop Loss: ${stop_loss}\nTP1: ${tp1}\nTP2: ${tp2}\nQuantity: {amount_bought} ETH\n\nNew Balance: ${balance}\n\n')

            trades += 1


        elif row['trigger_short'] == 1:

            print('SHORT')

            atr = round(row['ATR'],2)
            entry = round(row['close'],2)
            stop_loss = round(entry + (1*atr),2)
            tp1 = round(entry - atr,2)
            tp2 = round(entry - 2*atr,2)

            balance_split = balance*0.05
            amount_bought = (balance_split)/entry
            balance -= balance_split

            in_short = True

            print(f'ATR: ${atr}\nEntry Price: ${entry}\nStop Loss: ${stop_loss}\nTP1: ${tp1}\nTP2: ${tp2}\nQuantity: {amount_bought} ETH\n\nNew Balance: ${balance}\n\n')

            trades += 1


        else:

            continue
    
    elif in_long == True:

        if row['low'] <= stop_loss:

            print('LOSING TRADE')
            print(f'Old Balance: ${balance}')
            balance += amount_bought*stop_loss
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = False
            in_long = False

        elif tp1_hit == True and row['high'] >= tp2:

            print('WINNING TRADE')
            print(f'Old Balance: ${balance}')
            balance += amount_bought * tp2
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = False
            in_long = False
        
        elif tp1_hit == False and row['high'] >= tp1 and row['high'] < tp2:

            print('WINNING TRADE')
            print(f'Old Balance: ${balance}')
            amount_bought = amount_bought/2
            balance += amount_bought * tp1
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = True

        elif tp1_hit == False and row['high'] >= tp2:

            print('WINNING TRADE')
            print(f'Old Balance: ${balance}')
            amount_bought = amount_bought/2
            balance += (amount_bought * tp1) + (amount_bought * tp2)
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = False
            in_long = False
        
        else:
            
            continue

    else:

        if row['high'] >= stop_loss:

            print('LOSING TRADE')

            print(f'Old Balance: ${balance}')
            balance += (entry*amount_bought)+((entry*amount_bought)-(amount_bought*stop_loss))
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = False
            in_short = False

        elif row['low'] <= tp2 and tp1_hit == False:

            print('WINNING TRADE')

            print(f'Old Balance: ${balance}')
            amount_bought = amount_bought/2
            balance += (amount_bought * tp1) + (amount_bought * tp2)
            balance += (entry*amount_bought)+((entry*amount_bought)-(amount_bought*tp1))+((entry*amount_bought)-(amount_bought*tp2))
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = False
            in_short = False

        elif row['low'] <= tp2 and tp1_hit == True:

            print('WINNING TRADE')

            print(f'Old Balance: ${balance}')
            balance += (entry*amount_bought)+((entry*amount_bought)-(amount_bought*tp2))
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = False
            in_short = False

        elif tp1_hit == False and row['low'] <= tp1 and row ['low'] > tp2:

            print('WINNING TRADE')

            print(f'Old Balance: ${balance}')
            amount_bought = amount_bought/2
            balance += (entry*amount_bought)+((entry*amount_bought)-(amount_bought*tp1))
            print(f'New Balance: ${balance}\n\n')

            tp1_hit = True
        
        else:

            continue

        
print(f'Number Of Trades Executed: {trades}')

SHORT
ATR: $4.93
Entry Price: $1453.45
Stop Loss: $1458.38
TP1: $1448.52
TP2: $1443.59
Quantity: 0.34400908183976053 ETH

New Balance: $9500.0


LOSING TRADE
Old Balance: $9500.0
New Balance: $9998.30403522653


SHORT
ATR: $14.47
Entry Price: $1352.57
Stop Loss: $1367.04
TP1: $1338.1
TP2: $1323.63
Quantity: 0.36960394046986594 ETH

New Balance: $9498.388833465204


WINNING TRADE
Old Balance: $9498.388833465204
New Balance: $9751.020518855166


LOSING TRADE
Old Balance: $9751.020518855166
New Balance: $9998.30403522653


SHORT
ATR: $36.7
Entry Price: $1335.43
Stop Loss: $1372.13
TP1: $1298.73
TP2: $1262.03
Quantity: 0.37434773950063016 ETH

New Balance: $9498.388833465204


LOSING TRADE
Old Balance: $9498.388833465204
New Balance: $9984.565473186856


SHORT
ATR: $19.17
Entry Price: $1312.98
Stop Loss: $1332.15
TP1: $1293.81
TP2: $1274.64
Quantity: 0.3802253451380393 ETH

New Balance: $9485.337199527514


WINNING TRADE
Old Balance: $9485.337199527514
New Balance: $9738.595796290334


WIN